In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import random

np.random.seed(42)


In [3]:
cities = ["Mumbai", "Pune", "Bengaluru", "Delhi", "Hyderabad", "Chennai"]

customers = pd.DataFrame({
    "customer_id": range(1, 5001),
    "signup_date": pd.to_datetime("2023-01-01") + 
                   pd.to_timedelta(np.random.randint(0, 365, 5000), unit="D"),
    "city": np.random.choice(cities, 5000),
    "customer_type": np.random.choice(["New", "Repeat"], 5000, p=[0.6, 0.4])
})


In [4]:
order_dates = pd.to_datetime("2024-01-01") + \
              pd.to_timedelta(np.random.randint(0, 180, 25000), unit="D")

orders = pd.DataFrame({
    "order_id": range(1, 25001),
    "customer_id": np.random.randint(1, 5001, 25000),
    "order_date": order_dates,
    "order_amount": np.round(np.random.uniform(200, 2500, 25000), 2),
    "order_status": np.random.choice(
        ["Completed", "Cancelled"], 25000, p=[0.88, 0.12]
    ),
    "city": np.random.choice(cities, 25000)
})

orders["cancellation_reason"] = np.where(
    orders["order_status"] == "Cancelled",
    np.random.choice(["Delay", "Price", "Payment Failed"], 25000),
    None
)


In [5]:
payments = orders[["order_id", "order_date"]].copy()
payments["payment_id"] = range(1, 25001)
payments["payment_date"] = payments["order_date"] + \
                            pd.to_timedelta(np.random.randint(1, 10, 25000), unit="m")

payments["payment_method"] = np.random.choice(
    ["UPI", "Credit Card", "Debit Card", "Wallet"], 25000, p=[0.6, 0.2, 0.1, 0.1]
)

payments["payment_status"] = np.random.choice(
    ["Success", "Failed"], 25000, p=[0.93, 0.07]
)

payments["failure_reason"] = np.where(
    payments["payment_status"] == "Failed",
    np.random.choice(["Bank Error", "Timeout", "Insufficient Balance"], 25000),
    None
)


In [6]:
operations = orders[["order_id"]].copy()
operations["promised_time_min"] = 30
operations["actual_time_min"] = np.random.normal(28, 8, 25000).astype(int)

operations["sla_breached"] = operations["actual_time_min"] > 30

operations["ops_issue"] = np.where(
    operations["sla_breached"],
    np.random.choice(["Rider Shortage", "System Delay", "Inventory Issue"], 25000),
    None
)


In [11]:
# sample refunded orders (e.g., 10–15% of total orders)
refund_orders = orders.sample(frac=0.12, random_state=42).reset_index(drop=True)


In [12]:
# number of refunds
refund_sample_size = len(refund_orders)

refunds = pd.DataFrame({
    "refund_id": range(1, refund_sample_size + 1),
    "order_id": refund_orders["order_id"].values,
    "refund_amount": refund_orders["order_amount"].values,
    "refund_date": refund_orders["order_date"] +
                   pd.to_timedelta(
                       np.random.randint(1, 5, refund_sample_size),
                       unit="D"
                   ),
    "refund_reason": np.random.choice(
        ["Delay", "Payment Issue", "Quality Issue"],
        refund_sample_size
    )
})


In [13]:
customers.to_csv("customers.csv", index=False)
orders.to_csv("orders.csv", index=False)
payments.to_csv("payments.csv", index=False)
operations.to_csv("operations.csv", index=False)
refunds.to_csv("refunds.csv", index=False)


Python Risk Detection

In [1]:
import pandas as pd
import numpy as np


In [2]:
import mysql.connector

conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="P@wan123",
    database="risk_analysis"
)

orders = pd.read_sql("SELECT * FROM orders", conn)
orders["order_date"] = pd.to_datetime(orders["order_date"])


C:\Users\Pawan\AppData\Local\Temp\ipykernel_10728\953661266.py:10: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql("SELECT * FROM orders", conn)


In [3]:
orders.head()
orders.shape
orders.dtypes


order_id                       int64
customer_id                    int64
order_date             datetime64[s]
order_amount                 float64
order_status                     str
city                             str
cancellation_reason              str
dtype: object

In [4]:
orders_daily = orders.groupby(
    orders["order_date"].dt.date
).agg(
    total_orders=("order_id", "count"),
    cancelled_orders=("order_status", lambda x: (x == "Cancelled").sum()),
    total_order_amount=("order_amount", "sum")
).reset_index().rename(columns={"order_date": "date"})

orders_daily["cancel_rate"] = (
    orders_daily["cancelled_orders"] / orders_daily["total_orders"]
)


In [5]:
import pandas as pd
import numpy as np
import mysql.connector


In [6]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="P@wan123",
    database="risk_analysis"
)


In [7]:
payments = pd.read_sql("SELECT * FROM payments", conn)
refunds = pd.read_sql("SELECT * FROM refunds", conn)
operations = pd.read_sql("SELECT * FROM operations", conn)


C:\Users\Pawan\AppData\Local\Temp\ipykernel_10728\1045322806.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  payments = pd.read_sql("SELECT * FROM payments", conn)
C:\Users\Pawan\AppData\Local\Temp\ipykernel_10728\1045322806.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  refunds = pd.read_sql("SELECT * FROM refunds", conn)
C:\Users\Pawan\AppData\Local\Temp\ipykernel_10728\1045322806.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  operations = pd.read_sql("SELECT * FROM operations", conn)


In [8]:
payments["payment_date"] = pd.to_datetime(payments["payment_date"])
refunds["refund_date"] = pd.to_datetime(refunds["refund_date"])


In [9]:
payments.head()

,order_id,order_date,payment_id,payment_date,payment_method,payment_status,failure_reason
0,1,2024-06-17,1,2024-06-17 00:06:00,UPI,Success,
1,2,2024-06-20,2,2024-06-20 00:01:00,UPI,Failed,Insufficient Balance
2,3,2024-01-04,3,2024-01-04 00:08:00,UPI,Success,
3,4,2024-05-14,4,2024-05-14 00:01:00,UPI,Success,
4,5,2024-02-15,5,2024-02-15 00:09:00,Wallet,Success,


In [10]:
refunds.head()

,refund_id,order_id,refund_amount,refund_date,refund_reason
0,1,6869,1010.62,2024-04-20,Delay
1,2,24017,992.84,2024-02-02,Payment Issue
2,3,9669,1427.66,2024-04-12,Delay
3,4,13641,1371.55,2024-04-21,Quality Issue
4,5,14019,1599.65,2024-03-19,Payment Issue


In [11]:
operations.head()

,order_id,promised_time_min,actual_time_min,sla_breached,ops_issue
0,1,30,22,0,
1,2,30,36,1,System Delay
2,3,30,12,0,
3,4,30,39,1,Rider Shortage
4,5,30,25,0,


Daily PAYMENT FAILURE RATE

In [12]:
payments_daily = (
    payments
    .groupby(payments["payment_date"].dt.date)
    .agg(
        total_payments=("payment_id", "count"),
        failed_payments=("payment_status", lambda x: (x == "Failed").sum())
    )
    .reset_index()
    .rename(columns={"payment_date": "date"})
)

payments_daily["payment_fail_rate"] = (
    payments_daily["failed_payments"] / payments_daily["total_payments"]
)

payments_daily.head()


,date,total_payments,failed_payments,payment_fail_rate
0,2024-01-01,130,11,0.084615
1,2024-01-02,121,10,0.082645
2,2024-01-03,154,10,0.064935
3,2024-01-04,138,13,0.094203
4,2024-01-05,144,11,0.076389


Daily REFUND COUNT & REVENUE LOSS

In [13]:
refunds_daily = (
    refunds
    .groupby(refunds["refund_date"].dt.date)
    .agg(
        refund_count=("refund_id", "count"),
        refund_amount=("refund_amount", "sum")
    )
    .reset_index()
    .rename(columns={"refund_date": "date"})
)

refunds_daily.head()


,date,refund_count,refund_amount
0,2024-01-02,5,8588.54
1,2024-01-03,11,19406.95
2,2024-01-04,14,17491.27
3,2024-01-05,17,23446.43
4,2024-01-06,17,21160.50


SLA BREACH RATE (Operational Risk)

Because sla_breached is 0 / 1, the mean gives us a rate.

In [14]:
sla_daily = (
    operations
    .groupby("order_id")
    .agg(
        sla_breach_rate=("sla_breached", "mean")
    )
    .reset_index()
)

sla_daily.head()


,order_id,sla_breach_rate
0,1,0.0
1,2,1.0
2,3,0.0
3,4,1.0
4,5,0.0


MERGE INTO ONE DAILY RISK DATASET

In [15]:
daily_risk = (
    orders_daily
    .merge(payments_daily, on="date", how="left")
    .merge(refunds_daily, on="date", how="left")
)

daily_risk.fillna(0, inplace=True)
daily_risk.head()


,date,total_orders,cancelled_orders,total_order_amount,cancel_rate,total_payments,failed_payments,payment_fail_rate,refund_count,refund_amount
0,2024-01-01,130,19,172049.22,0.146154,130,11,0.084615,0.0,0.00
1,2024-01-02,121,13,166776.55,0.107438,121,10,0.082645,5.0,8588.54
2,2024-01-03,154,18,210216.29,0.116883,154,10,0.064935,11.0,19406.95
3,2024-01-04,138,13,187158.29,0.094203,138,13,0.094203,14.0,17491.27
4,2024-01-05,144,9,199077.10,0.062500,144,11,0.076389,17.0,23446.43


ABNORMAL CHANGE DETECTION STARTS HERE

In [16]:
daily_risk["date"] = pd.to_datetime(daily_risk["date"])
daily_risk = daily_risk.sort_values("date")


Rolling baselines for cancellation rate

In [17]:
daily_risk["cancel_rate_7d"] = (
    daily_risk["cancel_rate"]
    .rolling(window=7, min_periods=3)
    .mean()
)

daily_risk["cancel_rate_30d"] = (
    daily_risk["cancel_rate"]
    .rolling(window=30, min_periods=7)
    .mean()
)


Abnormal spike detection
We flag a risk when: 7-day avg > 1.3 × 30-day avg

In [18]:
daily_risk["cancel_risk_flag"] = (
    daily_risk["cancel_rate_7d"] >
    1.3 * daily_risk["cancel_rate_30d"]
)


Inspect flagged days

In [19]:
daily_risk[
    daily_risk["cancel_risk_flag"]
][
    ["date", "cancel_rate", "cancel_rate_7d", "cancel_rate_30d"]
].tail(10)


,date,cancel_rate,cancel_rate_7d,cancel_rate_30d


No rows showed above as no risk was detected to see if the logic is working check the below code

In [20]:
daily_risk[[
    "date",
    "cancel_rate",
    "cancel_rate_7d",
    "cancel_rate_30d"
]].tail(10)


,date,cancel_rate,cancel_rate_7d,cancel_rate_30d
170,2024-06-19,0.110236,0.112442,0.115562
171,2024-06-20,0.130435,0.117989,0.115518
172,2024-06-21,0.112676,0.122861,0.115528
173,2024-06-22,0.143939,0.120315,0.118834
174,2024-06-23,0.092308,0.122589,0.117212
175,2024-06-24,0.125000,0.121058,0.117883
176,2024-06-25,0.116667,0.118752,0.117955
177,2024-06-26,0.114286,0.119330,0.117184
178,2024-06-27,0.149660,0.122076,0.117065
179,2024-06-28,0.107143,0.121286,0.115199


PAYMENT FAILURE RISK DETECTION

Rolling baselines

In [21]:
daily_risk["payment_fail_7d"] = (
    daily_risk["payment_fail_rate"]
    .rolling(window=7, min_periods=3)
    .mean()
)

daily_risk["payment_fail_30d"] = (
    daily_risk["payment_fail_rate"]
    .rolling(window=30, min_periods=7)
    .mean()
)


Risk flag

In [22]:
daily_risk["payment_risk_flag"] = (
    daily_risk["payment_fail_7d"] >
    1.3 * daily_risk["payment_fail_30d"]
)


Inspect flagged days

In [23]:
daily_risk[
    daily_risk["payment_risk_flag"]
][
    ["date", "payment_fail_rate", "payment_fail_7d", "payment_fail_30d"]
].tail(10)


,date,payment_fail_rate,payment_fail_7d,payment_fail_30d


REFUND RISK DETECTION

Refund rate

In [24]:
daily_risk["refund_rate"] = (
    daily_risk["refund_count"] / daily_risk["total_orders"]
)


Rolling baselines

In [25]:
daily_risk["refund_rate_7d"] = (
    daily_risk["refund_rate"]
    .rolling(window=7, min_periods=3)
    .mean()
)

daily_risk["refund_rate_30d"] = (
    daily_risk["refund_rate"]
    .rolling(window=30, min_periods=7)
    .mean()
)


Refund rate risk flag

In [26]:
daily_risk["refund_risk_flag"] = (
    daily_risk["refund_rate_7d"] >
    1.3 * daily_risk["refund_rate_30d"]
)


Refund AMOUNT spike detection

In [27]:
daily_risk["refund_amt_7d"] = (
    daily_risk["refund_amount"]
    .rolling(window=7, min_periods=3)
    .mean()
)

daily_risk["refund_amt_30d"] = (
    daily_risk["refund_amount"]
    .rolling(window=30, min_periods=7)
    .mean()
)

daily_risk["refund_amt_risk_flag"] = (
    daily_risk["refund_amt_7d"] >
    1.5 * daily_risk["refund_amt_30d"]
)


In [28]:
daily_risk[
    (daily_risk["payment_risk_flag"]) |
    (daily_risk["refund_risk_flag"]) |
    (daily_risk["refund_amt_risk_flag"])
][
    [
        "date",
        "payment_risk_flag",
        "refund_risk_flag",
        "refund_amt_risk_flag",
        "payment_fail_rate",
        "refund_rate",
        "refund_amount"
    ]
].sort_values("date")


,date,payment_risk_flag,refund_risk_flag,refund_amt_risk_flag,payment_fail_rate,refund_rate,refund_amount


UNIFIED RISK SEVERITY SCORE

Numeric risk score

In [29]:
daily_risk["risk_score"] = (
    daily_risk["payment_risk_flag"].astype(int) * 1 +
    daily_risk["refund_risk_flag"].astype(int) * 2 +
    daily_risk["refund_amt_risk_flag"].astype(int) * 3
)


Risk severity label

In [30]:
def risk_level(score):
    if score >= 4:
        return "HIGH"
    elif score >= 2:
        return "MEDIUM"
    else:
        return "LOW"

daily_risk["risk_severity"] = daily_risk["risk_score"].apply(risk_level)


Final risk dashboard view

In [31]:
daily_risk[
    daily_risk["risk_severity"] != "LOW"
][
    [
        "date",
        "risk_score",
        "risk_severity",
        "payment_risk_flag",
        "refund_risk_flag",
        "refund_amt_risk_flag",
        "refund_amount"
    ]
].sort_values(["risk_score", "refund_amount"], ascending=False)


,date,risk_score,risk_severity,payment_risk_flag,refund_risk_flag,refund_amt_risk_flag,refund_amount


In [32]:
daily_risk["risk_severity"].value_counts()


risk_severity
LOW    180
Name: count, dtype: int64

Exporting for Power BI

In [33]:
daily_risk.to_csv("daily_risk_dashboard.csv", index=False)


In [34]:
city_daily = (
    orders.groupby(["order_date", "city"])
    .agg(
        total_orders=("order_id", "count"),
        cancelled_orders=("order_status", lambda x: (x == "Cancelled").sum()),
        revenue=("order_amount", "sum")
    )
    .reset_index()
)

city_daily["cancel_rate"] = (
    city_daily["cancelled_orders"] / city_daily["total_orders"]
)


In [35]:
refund_city = (
    refunds.merge(orders[["order_id", "city", "order_date"]],
                  on="order_id")
    .groupby(["order_date", "city"])
    .agg(
        refund_count=("refund_id", "count"),
        refund_amount=("refund_amount", "sum")
    )
    .reset_index()
)

city_risk = city_daily.merge(
    refund_city,
    on=["order_date", "city"],
    how="left"
).fillna(0)


In [36]:
city_risk.rename(columns={"order_date": "date"}, inplace=True)


In [37]:
city_risk.to_csv("city_risk_dashboard.csv", index=False)


In [39]:
orders.columns


Index(['order_id', 'customer_id', 'order_date', 'order_amount', 'order_status',
       'city', 'cancellation_reason'],
      dtype='str')

In [40]:
city_daily = (
    orders.groupby(["order_date", "city"])
    .agg(
        total_orders=("order_id", "count"),
        cancelled_orders=("order_status", lambda x: (x == "Cancelled").sum()),
        revenue=("order_amount", "sum")
    )
    .reset_index()
)

city_daily["cancel_rate"] = (
    city_daily["cancelled_orders"] / city_daily["total_orders"]
)


In [41]:
refund_city = (
    refunds.merge(
        orders[["order_id", "city", "order_date"]],
        on="order_id",
        how="left"
    )
    .groupby(["order_date", "city"])
    .agg(
        refund_count=("refund_id", "count"),
        refund_amount=("refund_amount", "sum")
    )
    .reset_index()
)

city_risk = city_daily.merge(
    refund_city,
    on=["order_date", "city"],
    how="left"
).fillna(0)

city_risk.rename(columns={"order_date": "date"}, inplace=True)


In [42]:
city_risk.to_csv("city_risk_dashboard.csv", index=False)
